In [82]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
import hvplot.pandas
import numpy as np
import urllib
import time
import requests

In [74]:
def status_cfs(s):
    ti = min(s)
    tf = max(s)
    return 'Collected {} records, from {}/{} up to {}/{}.'.format(len(s), ti.month, ti.year, tf.month, tf.year)
    
    
def get_tempe_cfs(dt_id='occ_dt'):
    url = 'http://data-tempegov.opendata.arcgis.com/datasets/ba76da19a5274301bb8a010483aec14a_0.csv'
    df = pd.read_csv(url, parse_dates=[dt_id])
    print(status_cfs(df[dt_id]))
    df.rename(columns={dt_id: 'call_dt'}, inplace=True)
    
    return df


def get_mesa_cfs(dt_id='Creation Datetime'):
    url = 'https://data.mesaaz.gov/api/views/4k95-x7aw/rows.csv'
    df = pd.read_csv(url, parse_dates=[dt_id])
    print(status_cfs(df[dt_id]))
    df.rename(columns={dt_id: 'call_dt'}, inplace=True)
    
    return df


def get_gilbert_cfs(dt_id='EventDate'):
    url = 'https://opendata.arcgis.com/datasets/2dcb4c20c9a444f2a5d3dca180b651c9_2.csv'
    df = pd.read_csv(url, parse_dates=[dt_id])
    print(status_cfs(df[dt_id]))
    df.rename(columns={dt_id: 'call_dt'}, inplace=True)
    
    return df


def get_phoenix_cfs(dt_id='CALL_RECEIVED'):
    url = 'https://www.phoenixopendata.com/dataset/64a60154-3b2d-4583-8fb5-6d5e1b469c28/resource/1d536ee6-7ffb-49c3-bffe-5cdd98a3c97e/download/calls-for-service_calls-for-service_callsforservice.csv'
    df = pd.read_csv(url, parse_dates=[dt_id])
    print(status_cfs(df[dt_id]))
    df.rename(columns={dt_id: 'call_dt'}, inplace=True)
    
    return df


def get_scottsdale_cfs(dt_id='Create Date', ext='csv'):
    base_url = 'http://data.scottsdaleaz.gov:8000/spd_PDCallsForServic_'
    i = 1
    for year in range(2017, 2020):
        for month in range(1, 13):
            yearmonth = '{year}_{month}.{ext}'.format(year=year, month=month, ext=ext)
            url = base_url + yearmonth
            if i:
                try:
                    df = pd.read_csv(url, parse_dates=[dt_id])
                    i -= 1
                    month0, year0 = month, year
                except:
                    print(url, 'seems to be invalid.')
            else:
                try:
                    df2 = pd.read_csv(url, parse_dates=[dt_id])
                    df = pd.concat([df, df2])
                except:
                    break
         
    print(status_cfs(df[dt_id]))
    df.rename(columns={dt_id: 'call_dt'}, inplace=True)
    
    return df


def get_chandler_cfs(dt_id='Call Received'):
    base_url = 'https://data.chandlerpd.com/catalog/calls-for-service-'
    i = 1
    dfs = dict()
    for year in range(2013, 2020):
        year_id = '{year}/download/csv'.format(year=str(year))
        url = base_url + year_id
        if i:
            df = pd.read_csv(url, parse_dates=[dt_id])
            i -= 1
        else:
            df2 = pd.read_csv(url, parse_dates=[dt_id])
            df = pd.concat([df, df2])
        print('Finished', year)
    
    print(status_cfs(df[dt_id]))
    df.rename(columns={dt_id: 'call_dt'}, inplace=True)
    
    return df

In [75]:
chandler = get_chandler_cfs()

/Users/tjs/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3185: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  if (yield from self.run_code(code, result)):


Finished 2013
Finished 2014
Finished 2015
Finished 2016
Finished 2017
Finished 2018
Finished 2019
Collected 910626 records, from 1/2013 up to 2/2019.


In [76]:
chandler.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 910626 entries, 0 to 13668
Data columns (total 24 columns):
ID                             910626 non-null int64
Number                         910626 non-null object
call_dt                        910626 non-null datetime64[ns]
Place Name                     313339 non-null object
Address                        910625 non-null object
City                           908692 non-null object
District                       910626 non-null int64
Beat                           910626 non-null int64
Latitude                       892419 non-null object
Longitude                      891741 non-null object
Priority                       910626 non-null int64
How Received                   910614 non-null object
Reported As                    910626 non-null object
Closed As                      910569 non-null object
Cleared By                     910625 non-null object
Dispatched                     910626 non-null object
En Route              

In [80]:
chandler.dropna(subset=['Latitude', 'Longitude', 'Closed As'], inplace=True)
coords = pd.Series(zip(chandler.Longitude.astype(np.float), chandler.Latitude.astype(np.float)))
chandler['Coordinates'] = coords.apply(Point)

chandler.rename(columns={'Closed As': 'final_code'}, inplace=True)
ch_acc = chandler[chandler.final_code.str.contains('Accident')]
ch_acc['PD'] = 'Chandler'
ch_acc = ch_acc.loc[:, ['call_dt', 'final_code', 'Coordinates', 'PD']]

/Users/tjs/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [83]:
ch_acc.head()

,call_dt,final_code,Coordinates,PD
34,2013-09-09 17:47:05,Accident/Non-Injuries,POINT (-111.8486709594727 33.34972381591797),Chandler
114,2013-01-03 18:10:07,Accident/Non-Injuries,POINT (-111.8619766235352 33.33873748779297),Chandler
133,2013-09-09 22:54:09,Accident/Non-Injuries,POINT (-111.8355865478516 33.2962646484375),Chandler
215,2013-09-10 07:13:55,Accident/Non-Injuries,POINT (-111.8134078979492 33.27713775634766),Chandler
220,2013-09-10 07:18:56,Accident/Non-Injuries,POINT (-111.8822250366211 33.29138946533203),Chandler


In [53]:
total = ch_acc.shape[0]
hits = ch_acc[ch_acc['final_code'].str.contains('Hit')].shape[0]

ch_acc.to_csv('chandler_accidents.csv')

In [66]:
print(str(round(hits/total* 100, 3)) + '%', 'of accident calls reported to Chandler PD are Hit & Run.')

15.496% of accident calls reported to Chandler PD are Hit & Run.


In [62]:
chandler.head()

,ID,Number,call_dt,Place Name,Address,City,District,Beat,Latitude,Longitude,...,En Route,On Scene,Response Time,911 to Received Time,Dispatch to Enroute Time,Call Received to Queue Time,Call Queue Time,Call Travel Time,Coordinates,PD
0,316229,2013-1,2013-01-01 00:02:01,Chandler Water Treatment Facil,1400 E Pecos Rd,Chandler,5,12,33.289249420166016,-111.81481170654297,...,2013-01-01 00:07:18,2013-01-01 00:16:28,867,0,0,0,317,550,POINT (-111.814811706543 33.28924942016602),Chandler
1,166758,2013-10,2013-01-01 00:07:19,NaN,1800 W Alamo Dr,Chandler,2,4,33.348194122314453,-111.87547302246094,...,0000-00-00 00:00:00,0000-00-00 00:00:00,0,0,0,285,0,0,POINT (-111.8754730224609 33.34819412231445),Chandler
3,166846,2013-1000,2013-01-03 18:11:01,NaN,1800 W Canyon Wy,Chandler,6,13,33.238456726074219,-111.87477111816406,...,2013-01-03 18:14:19,2013-01-03 18:25:44,924,41,0,5,193,685,POINT (-111.8138732910156 33.32269287109375),Chandler
4,166940,2013-10000,2013-01-27 21:59:45,NaN,1400 E Park Av,Chandler,4,11,33.32269287109375,-111.81387329101562,...,2013-01-27 22:09:53,2013-01-27 22:12:06,748,7,5,68,535,133,POINT (-111.8764495849609 33.34391021728516),Chandler
5,267483,2013-100000,2013-09-09 16:56:33,NaN,N Dobson Rd & W Shawnee Dr,Chandler,2,4,33.343910217285156,-111.87644958496094,...,0000-00-00 00:00:00,0000-00-00 00:00:00,0,0,0,21,0,0,POINT (-111.8524703979492 33.30901718139648),Chandler


In [63]:
chandler = chandler.loc[:, ['call_dt', 'Closed As', 'Coordinates', 'PD']]

In [66]:
chandler.rename(columns={'Closed As': 'final_code'}, inplace=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 890915 entries, 0 to 12766
Data columns (total 4 columns):
call_dt        890915 non-null datetime64[ns]
final_code     890860 non-null object
Coordinates    890915 non-null object
PD             890915 non-null object
dtypes: datetime64[ns](1), object(3)
memory usage: 34.0+ MB


In [8]:
#ch_acc = chandler[chandler['Reported As'].str.contains('Accident')]

In [85]:
scottsdale = get_scottsdale_cfs()

Collected 444695 records, from 1/2017 up to 1/2019.


In [93]:
sc_acc = scottsdale[scottsdale['CFS_Type'].str.contains('96')]

In [113]:
print(str(round(sc_acc[sc_acc.CFS_Type.str.contains('HR')].shape[0] / sc_acc.shape[0] * 100, 3)) + '%')

11.433%


In [10]:
#sct_acc = scottsdale[scottsdale.CFS_English.str.contains('Accident')]

In [107]:
tempe = get_tempe_cfs()

/Users/tjs/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3185: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  if (yield from self.run_code(code, result)):


Collected 934409 records, from 1/2012 up to 1/2019.


In [118]:
tempe.dropna(subset=['final_case_type'], inplace=True)

In [119]:
tempe_acc = tempe[tempe.final_case_type.str.contains('96')]

In [134]:
tempe_acc.drop(tempe_acc[tempe_acc.final_case_type.str.contains('964')].index, inplace=True)

In [135]:
tempe_acc[tempe_acc.final_case_type.str.contains('H')].shape[0] / tempe_acc.shape[0]

0.2081152664477169

In [108]:
mesa = get_mesa_cfs()

Collected 284858 records, from 1/2017 up to 12/2018.


In [109]:
gilbert = get_gilbert_cfs()

Collected 642554 records, from 11/2006 up to 1/2019.


In [99]:
phoenix = get_phoenix_cfs()

Collected 2203183 records, from 11/2015 up to 1/2019.


In [16]:
total = len(tempe) + len(scottsdale) + len(mesa) + len(phoenix) + len(gilbert) + len(chandler)
total

5415915

In [101]:
hg = phoenix[phoenix.GRID == 'BI37']

In [102]:
hg.reset_index(inplace=True)

In [106]:
hg.FINAL_CALL_TYPE

0                       CRIMINAL DAMAGE
1                           TRESPASSING
2                        BURGLARY ALARM
3             SOLICITATION DOOR TO DOOR
4                        BURGLARY ALARM
5          SUSPICIOUS PERSON IN VEHICLE
6                         CHECK WELFARE
7                                 FIGHT
8                   ARMED ROBBERY ALARM
9                     SUSPICIOUS PERSON
10               ACCIDENT WITH INJURIES
11         SUSPICIOUS PERSON IN VEHICLE
12                           PR CONTACT
13               LOUD NOISE DISTURBANCE
14                        CHECK WELFARE
15                    SUSPICIOUS PERSON
16                    SUSPICIOUS PERSON
17                  RECOVERY OF VEHICLE
18                        CHECK WELFARE
19                        CHECK WELFARE
20                 CIVIL MATTER/STANDBY
21               LOUD NOISE DISTURBANCE
22                 BURGLARY RESIDENTIAL
23                     NEIGHBOR DISPUTE
24               LOUD NOISE DISTURBANCE


In [123]:
df = get_scottsdale_cfs()

Collected 444695 records, from 1/2017 up to 1/2019.


In [121]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 444695 entries, 2017-01-15 12:33:02 to 2019-01-20 01:44:46
Data columns (total 13 columns):
Common Event ID              444695 non-null int64
CFS Number                   444695 non-null object
Street Directional Prefix    335623 non-null object
Street Name                  336881 non-null object
Street Type                  320650 non-null object
Cross Street #1              194580 non-null object
Cross Street #2              201810 non-null object
Call Type                    444695 non-null object
Call Sub-type                444695 non-null object
Call Type English            440100 non-null object
Call Sub-type English        139877 non-null object
CFS_Type                     444695 non-null object
CFS_English                  444695 non-null object
dtypes: int64(1), object(12)
memory usage: 47.5+ MB


In [168]:
def get_tempe_cfs():
    url = 'http://data-tempegov.opendata.arcgis.com/datasets/ba76da19a5274301bb8a010483aec14a_0.csv'
    tempe = pd.read_csv(url, parse_dates=['occ_dt'])
    
    return tempe

In [169]:
def get_mesa_cfs():
    url = 'https://data.mesaaz.gov/api/views/4k95-x7aw/rows.csv'
    mesa = pd.read_csv(url, parse_dates=['Creation Datetime'])
    
    return mesa

In [170]:
def get_gilbert_cfs():
    url = 'https://opendata.arcgis.com/datasets/2dcb4c20c9a444f2a5d3dca180b651c9_2.csv'
    gilbert = pd.read_csv(url, parse_dates=['EventDate'])
    
    return gilbert

In [ ]:
tempe = get_tempe_cfs()
mesa = get_mesa_cfs()
gilbert = get_gilbert_cfs()

CPU times: user 1.49 s, sys: 273 ms, total: 1.77 s
Wall time: 7.49 s


In [163]:
mesa.head()

,Event Number,Creation Datetime,Creation Month,Creation Year,Event Type Code,Event Type Description,Call Priority,Event Address,Street Number,Street Direction,Street Name,Street Type,Latitude,Longitude,Dispatch Group,Division,Response Time Seconds,Response Time Minutes,Location 1
0,P20170010004,01/01/2017 12:02:00 AM,Jan,2017,900,WELFARE CHECK,P3,400 Block of N LESUEUR MESA,400.0,N,LESUEUR,NaN,33.422611,-111.820437,P2,Central,69,1.15,"(33.422611, -111.820437)"
1,P20170010005,01/01/2017 12:03:00 AM,Jan,2017,921,PROWLER,P1,1200 Block of W 6TH DR MESA,1200.0,W,6TH,DR,33.402687,-111.857332,P1,Fiesta,427,7.12,"(33.402687, -111.857332)"
2,P20170010008,01/01/2017 12:05:00 AM,Jan,2017,666,SUICIDE,P1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,P4,Superstition,558,9.30,NaN
3,P20170010010,01/01/2017 12:06:00 AM,Jan,2017,961,ACCIDENT,P1,1600 Block of S DOBSON RD MESA,1600.0,S,DOBSON,RD,33.384051,-111.875400,P1,Fiesta,43,0.72,"(33.384051, -111.8754)"
4,P20170010015,01/01/2017 12:09:00 AM,Jan,2017,421,SHOTS FIRED,P1,1700 Block of S SOSSAMAN RD MESA,1700.0,S,SOSSAMAN,RD,33.383873,-111.667177,P4,Superstition,151,2.52,"(33.383873, -111.667177)"


In [156]:
len(mesa)

TypeError: object of type 'JsonReader' has no len()

In [61]:
df = pd.read_csv('callsforservice.csv')
df.head()

,INCIDENT_NUM,DISP_CODE,DISPOSITION,FINAL_RADIO_CODE,FINAL_CALL_TYPE,CALL_RECEIVED,HUNDREDBLOCKADDR,GRID
0,201502108490,N,NO ACTION REQUIRED,647V,SUSPICIOUS PERSON IN VEHICLE,11/2/2015 9:16:27AM,154XX N 29TH ST,DF34
1,201502108749,N,NO ACTION REQUIRED,418T,TRESPASSING,11/2/2015 9:58:57AM,96XX N 10TH AVE,CH26
2,201502108648,O,OTHER (SUPP/FALSE ALARM/FI/LOUD PARTY),459A,BURGLARY ALARM,11/2/2015 9:42:42AM,38XX E WATKINS ST,AH36B
3,201502108757,N,NO ACTION REQUIRED,418T,TRESPASSING,11/2/2015 10:02:11AM,36XX W CAMELBACK RD,BI20
4,201502108592,N,NO ACTION REQUIRED,900,CHECK WELFARE,11/2/2015 9:30:53AM,40XX N 44TH AVE,BG18


In [62]:
with open('apikey.txt', 'r') as door:
    key = door.read()

In [63]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2192756 entries, 0 to 2192755
Data columns (total 8 columns):
INCIDENT_NUM        int64
DISP_CODE           object
DISPOSITION         object
FINAL_RADIO_CODE    object
FINAL_CALL_TYPE     object
CALL_RECEIVED       object
HUNDREDBLOCKADDR    object
GRID                object
dtypes: int64(1), object(7)
memory usage: 133.8+ MB


In [64]:
# tst = df.iloc[:500, :]
# tst.head()

In [65]:
df['HUNDREDBLOCKADDR'] = df.HUNDREDBLOCKADDR.str.replace('XX', '00')
df.dropna(inplace=True)
df = df[df.FINAL_RADIO_CODE.isin(['961', '962', '963', '961H', '962H', '963H'])]
df.loc[:, 'CALL_RECEIVED'] = pd.to_datetime(df.CALL_RECEIVED)
df.set_index('CALL_RECEIVED', inplace=True)

In [66]:
df['month'] = df.index.month
df['day'] = df.index.dayofweek
df['hour'] = df.index.hour

In [67]:
df.head()

,INCIDENT_NUM,DISP_CODE,DISPOSITION,FINAL_RADIO_CODE,FINAL_CALL_TYPE,HUNDREDBLOCKADDR,GRID,month,day,hour
CALL_RECEIVED,,,,,,,,,,
2015-11-01 01:01:07,201502100315,N,NO ACTION REQUIRED,961H,HIT & RUN ACCIDENT NO INJURY,S 67TH AVE & W LOWER BUCKEYE RD,AG13,11,6,1
2015-11-02 08:04:44,201502108062,A,ACCIDENT REPORT,962,ACCIDENT WITH INJURIES,N 49TH DR & W ESCUDA DR,EA17,11,0,8
2015-11-01 10:09:50,201502102088,N,NO ACTION REQUIRED,961,ACCIDENT NO INJURIES,E PARADISE LN & N 32ND ST,DF35,11,6,10
2015-11-01 10:35:06,201502102198,N,NO ACTION REQUIRED,961,ACCIDENT NO INJURIES,4000 W CAMELBACK RD,BI19A,11,6,10
2015-11-01 18:03:38,201502104894,N,NO ACTION REQUIRED,961,ACCIDENT NO INJURIES,3700 W SIERRA ST,DA20,11,6,18


In [68]:
df['ishitrun'] = df.FINAL_RADIO_CODE.isin(['961H', '962H', '963H']).astype(np.int)

In [69]:
hracs = df[df.ishitrun == 1]
lacs = df[df.ishitrun == 0]

In [70]:
lacs.month.hvplot.hist(color='navy').options(alpha=0.7) * hracs.month.hvplot.hist(color='red').options(alpha=0.7)

:Overlay
   .Histogram.I  :Histogram   [month]   (month_count)
   .Histogram.II :Histogram   [month]   (month_count)

In [71]:
lacs.day.hvplot.hist(color='navy').options(alpha=0.7) * hracs.day.hvplot.hist(color='red').options(alpha=0.7)

:Overlay
   .Histogram.I  :Histogram   [day]   (day_count)
   .Histogram.II :Histogram   [day]   (day_count)

In [72]:
lacs.hour.hvplot.hist(color='navy').options(alpha=0.7) * hracs.hour.hvplot.hist(color='red').options(alpha=0.7)

:Overlay
   .Histogram.I  :Histogram   [hour]   (hour_count)
   .Histogram.II :Histogram   [hour]   (hour_count)

In [73]:
lacs.DISPOSITION.value_counts() / len(lacs) * 100

ACCIDENT REPORT                           61.493233
NO ACTION REQUIRED                        31.057541
DEPARTMENTAL REPORT                        6.999781
OTHER (SUPP/FALSE ALARM/FI/LOUD PARTY)     0.228210
TRAFFIC CITATION                           0.221235
Name: DISPOSITION, dtype: float64

In [74]:
hracs.DISPOSITION.value_counts() / len(hracs) * 100

NO ACTION REQUIRED                        51.400935
DEPARTMENTAL REPORT                       24.767106
ACCIDENT REPORT                           23.278724
OTHER (SUPP/FALSE ALARM/FI/LOUD PARTY)     0.478281
TRAFFIC CITATION                           0.074954
Name: DISPOSITION, dtype: float64

In [75]:
lacs.FINAL_CALL_TYPE.value_counts() / len(lacs) * 100

ACCIDENT NO INJURIES      81.209017
ACCIDENT WITH INJURIES    18.576725
ACCIDENT FATALITY          0.214259
Name: FINAL_CALL_TYPE, dtype: float64

In [76]:
hracs.FINAL_CALL_TYPE.value_counts() / len(hracs) * 100

HIT & RUN ACCIDENT NO INJURY        93.550344
HIT & RUN ACCIDENT WITH INJURIES     6.356855
HIT & RUN  FATALITY                  0.092801
Name: FINAL_CALL_TYPE, dtype: float64

In [77]:
len(hracs) / (len(hracs) + len(lacs))

0.2182638299198367

In [78]:
df.GRID.value_counts()

BG27     705
BA27     624
BG13     603
BC19A    590
BI23B    569
DG29     553
CI22     529
BC17A    528
CA21     522
BE11     519
BG19     516
BG17     516
BE31     510
AB16     503
BI31B    485
BE38     484
BI29     484
BD11     483
BG29     479
BG11     474
BE17     460
BI32     458
BC13A    458
BE35     449
CA25     434
BC29A    426
BG21A    418
CA23B    416
BI21     414
CG21     404
        ... 
HA16       1
EH14       1
IH29       1
EF39       1
EE28       1
HA17       1
JA28       1
DE30       1
EI18       1
JA27       1
AJ10       1
FI37       1
CI20B      1
AF08       1
EE40       1
JC22       1
JD26       1
EH34       1
AE14       1
EI15       1
FG42       1
AC10       1
CC18       1
GC18       1
FC13       1
EF34       1
GF12       1
EI34       1
IC12       1
EI17       1
Name: GRID, Length: 1669, dtype: int64